# Commute Carbon Footprint Prediction with PyTorch

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

torch.manual_seed(42)
np.random.seed(42)


## 2. Load and Preprocess Data

In [ ]:
data_file = 'commute_data.csv'
try:
    df = pd.read_csv(data_file)
    print(f"Successfully loaded data from {data_file}")
except FileNotFoundError:
    print(f"Error: File {data_file} not found.")
    raise
except Exception as e:
    print(f"Error loading data: {str(e)}")
    raise

df['avg_speed'] = df['distance_km'] / (df['trip_duration'] / 60)
df['co2_per_km'] = df['co2_emissions_kg'] / df['distance_km']
df['direction_binary'] = (df['trip_direction'] == 'Home to Campus').astype(int)
df['departure_hour'] = pd.to_datetime(df['departure_time'], format='%H:%M').dt.hour
traffic_map = {0: 'Low', 1: 'Moderate', 2: 'High'}
df['traffic_text'] = df['traffic_condition'].map(traffic_map)

df.head()


## 3. Summary Statistics

In [ ]:
print(f"Loaded {len(df)} commute records")
print(f"Average CO2 emissions: {df['co2_emissions_kg'].mean():.2f} kg")
traffic_summary = df.groupby('traffic_text')['co2_emissions_kg'].agg(['mean', 'count'])
traffic_summary


## 4. Define Neural Network

In [ ]:
class CarbonFootprintNN(nn.Module):
    def __init__(self, input_dim):
        super(CarbonFootprintNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 32)
        self.layer2 = nn.Linear(32, 16)
        self.layer3 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x


## 5. Prepare Dataset

In [ ]:
features = ['traffic_condition', 'trip_duration', 'distance_km', 'fuel_efficiency_l_per_100km']
X = df[features].values
y = df['co2_emissions_kg'].values.reshape(-1, 1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)


## 6. Train Model

In [ ]:
model = CarbonFootprintNN(X.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

num_epochs = 500
train_losses, test_losses = [], []

for epoch in range(num_epochs):
    model.train()
    train_loss = sum(criterion(model(xb), yb).item() for xb, yb in train_loader) / len(train_loader)
    train_losses.append(train_loss)

    model.eval()
    with torch.no_grad():
        test_loss = sum(criterion(model(xb), yb).item() for xb, yb in test_loader) / len(test_loader)
    test_losses.append(test_loss)

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")


## 7. Evaluation

In [ ]:
model.eval()
all_predictions, all_targets = [], []

with torch.no_grad():
    for xb, yb in test_loader:
        pred = model(xb)
        all_predictions.extend(pred.numpy().flatten())
        all_targets.extend(yb.numpy().flatten())

all_predictions = np.array(all_predictions)
all_targets = np.array(all_targets)

mse = np.mean((all_predictions - all_targets) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(all_predictions - all_targets))
r2 = 1 - (np.sum((all_targets - all_predictions)**2) / np.sum((all_targets - np.mean(all_targets))**2))

print(f"MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")


## 8. Visualizations

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Learning Curves')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 6))
plt.scatter(all_targets, all_predictions, alpha=0.6)
plt.plot([min(all_targets), max(all_targets)], [min(all_targets), max(all_targets)], 'r--')
plt.xlabel('Actual CO2 Emissions (kg)')
plt.ylabel('Predicted CO2 Emissions (kg)')
plt.title('Predicted vs Actual CO2 Emissions')
plt.grid(True)
plt.show()


## 9. Save Model

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'input_dim': X.shape[1],
    'features': features,
    'scaler': scaler
}, 'commute_carbon_pytorch_model.pt')
print("Model saved.")
